In [1]:
import sqlite3
import pandas as pd

# SQLiteの場合の例。別のDBを利用される場合はSQLAlchemyなどをご利用ください。
db_path = 'jobs.db'  # ご自身のDBファイルに置き換えてください
conn = sqlite3.connect(db_path)

# テーブル名は仮にjob_tableとして、必要に応じて修正してください
query = """
SELECT
    title,
    salary,
    company,
    location,
    welcomed_skills,
    employment_type
FROM jobs.db
"""

df = pd.read_sql_query(query, conn)
conn.close()

# 取得したデータの先頭5行を確認
df.head()

DatabaseError: Execution failed on sql '
SELECT
    title,
    salary,
    company,
    location,
    welcomed_skills,
    employment_type
FROM jobs.db
': no such table: jobs.db

In [ ]:
import re
from collections import Counter

# タイトルのテキストを単語に分割（例として単純なsplitや正規表現を使用）
# 日本語分かち書きが必要な場合はJanomeやMeCabなど適切なライブラリを使用する
word_list = []
for t in df['title']:
    # 英字と数字と日本語(ひらがな・カタカナ・漢字)を考慮した単純な正規表現の例
    # Pythonのバージョンや環境によってはうまく動かない場合があるので注意
    words = re.findall(r'[a-zA-Z0-9ぁ-んァ-ン一-龥]+', str(t))
    word_list.extend(words)

# ワードの出現回数上位5つを抽出
counter = Counter(word_list)
top5_words = [w[0] for w in counter.most_common(5)]
top5_words

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# タイトルにtop5_wordsが含まれているかどうかをフラグ化する
for word in top5_words:
    df[f'has_{word}'] = df['title'].astype(str).apply(lambda x: 1 if word in x else 0)

# グラフ表示のために各ワードが含まれるデータの平均年収を計算
avg_salaries = []
for word in top5_words:
    avg_salaries.append(df[df[f'has_{word}'] == 1]['salary'].mean())

plt.figure(figsize=(8, 6))
sns.barplot(x=top5_words, y=avg_salaries)
plt.title('Top 5 Words in Title and Average Salary')
plt.xlabel('Word')
plt.ylabel('Average Salary')
plt.show()

In [ ]:
company_counts = df['company'].value_counts()
top5_companies = company_counts.index[:5]

# 上位5社ごとに平均年収を計算
company_salary = df[df['company'].isin(top5_companies)].groupby('company')['salary'].mean()

plt.figure(figsize=(8, 6))
sns.barplot(x=company_salary.index, y=company_salary.values)
plt.title('Top 5 Companies and Average Salary')
plt.xlabel('Company')
plt.ylabel('Average Salary')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
location_counts = df['location'].value_counts()
top5_locations = location_counts.index[:5]

# 上位5つの勤務地ごとに平均年収を計算
location_salary = df[df['location'].isin(top5_locations)].groupby('location')['salary'].mean()

plt.figure(figsize=(8, 6))
sns.barplot(x=location_salary.index, y=location_salary.values)
plt.title('Top 5 Locations and Average Salary')
plt.xlabel('Location')
plt.ylabel('Average Salary')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
from collections import Counter

all_skills = []
for s in df['skills']:
    if pd.notnull(s):
        # カンマ区切り又はスペース区切りなどで split する例
        splitted = re.split(r'[,\s]+', s.strip())
        all_skills.extend(splitted)

skill_counter = Counter(all_skills)
top5_skills = [skill for skill, count in skill_counter.most_common(5)]
top5_skills

In [ ]:
# スキルのフラグ列を作成
for skill in top5_skills:
    df[f'has_{skill}'] = df['skills'].astype(str).apply(lambda x: 1 if skill in x else 0)

skill_avg_salaries = []
for skill in top5_skills:
    skill_avg_salaries.append(df[df[f'has_{skill}'] == 1]['salary'].mean())

plt.figure(figsize=(8, 6))
sns.barplot(x=top5_skills, y=skill_avg_salaries)
plt.title('Top 5 Skills and Average Salary')
plt.xlabel('Skill')
plt.ylabel('Average Salary')
plt.show()

In [ ]:
employment_salary = df.groupby('employment_type')['salary'].mean()

plt.figure(figsize=(8, 6))
sns.barplot(x=employment_salary.index, y=employment_salary.values)
plt.title('Employment Type and Average Salary')
plt.xlabel('Employment Type')
plt.ylabel('Average Salary')
plt.show()

In [ ]:
# タイトル上位5ワードフラグとsalaryのみを抽出
cols_for_corr = [f'has_{w}' for w in top5_words] + ['salary']
corr_df_title = df[cols_for_corr].corr()

# 相関行列を表示
corr_df_title

In [ ]:
df['company_top5_or_other'] = df['company'].apply(lambda x: x if x in top5_companies else 'Other')
df_company_dummies = pd.get_dummies(df['company_top5_or_other'], prefix='company')

# 年収と企業ダミー変数をまとめる
df_for_corr_company = pd.concat([df_company_dummies, df['salary']], axis=1)

# 相関行列
corr_df_company = df_for_corr_company.corr()
corr_df_company

In [ ]:
df['location_top5_or_other'] = df['location'].apply(lambda x: x if x in top5_locations else 'Other')
df_location_dummies = pd.get_dummies(df['location_top5_or_other'], prefix='location')

df_for_corr_location = pd.concat([df_location_dummies, df['salary']], axis=1)

corr_df_location = df_for_corr_location.corr()
corr_df_location

In [ ]:
cols_for_corr_skills = [f'has_{skill}' for skill in top5_skills] + ['salary']
corr_df_skills = df[cols_for_corr_skills].corr()

corr_df_skills

In [ ]:
df_employment_dummies = pd.get_dummies(df['employment_type'], prefix='emp_type')
df_for_corr_employment = pd.concat([df_employment_dummies, df['salary']], axis=1)

corr_df_employment = df_for_corr_employment.corr()
corr_df_employment